## Gemini 1.5 Flash를 활용한 챗봇

- 무료 api
- 비율 한도: 분당 요청 15개, 1백만 TPM(분당 토큰), 일 요청 수 1500회
- 컨택스트 캐싱은 시간당 최대 100만 토큰

---

### 프로세스

1. 데이터베이스, 테이블, 컬럼, 컬럼 매핑 규칙과 쿼리로 변환하는 규칙, 예시를, 제약조건에 대한 정보를 담은 프롬프트를 gemini에 제공한다. 그러면 사용자 입력을 프롬프트에 맞게 gemini가 sql문으로 변환하여 전달한다.

2. db와 연결하여 위에서 생성된 sql문으로 데이터를 df으로 읽어온다.

3. df을 텍스트로 변환한 뒤, 이를 gemini에게 질문에 맞는 자연어로 변환해달라는 프롬프트로 요청한다.

4. 변환 완료! ask함수에 원하는 질문을 입력하면 된다.

(langchain이나 RAG 쓰지 않고 api만 활용.)

---

In [2]:
#import
import google.generativeai as genai
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
genai.configure(api_key='your-api-key')

def get_gemini_sql(question, prompt):
    """
    Gemini를 사용하여 한국어 질문을 SQL 쿼리로 변환
    
    Args:
        question (str): 한국어 자연어 질문
        prompt (list): 프롬프트 지침
    
    Returns:
        str: 생성된 SQL 쿼리
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    sql = model.generate_content([prompt[0], question])
    return " ".join(sql.text.split('\n')[1:-2])

def convert_result_to_natural_language(question, df):
    """
    SQL 쿼리 결과를 자연어로 해석
    
    Args:
        question (str): 원본 한국어 질문
        df (pd.DataFrame): 쿼리 결과 데이터프레임
    
    Returns:
        str: 자연어로 된 결과 설명
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    df_text = df.to_string(index=False)
    
    # 자연어 변환 프롬프트
    natural_language_prompt = f"""
    다음 SQL 쿼리 결과를 사용자가 이해하기 쉬운 한국어로 설명해주세요.
    
    원본 질문: {question}
    쿼리 결과:
    {df_text}
    
    결과를 간결하고 명확하게 해석해주세요. 데이터의 핵심 포인트를 강조하고, 
    사용자가 쉽게 이해할 수 있는 방식으로 설명해주세요.
    """
    
    response = model.generate_content(natural_language_prompt)
    
    return response.text

def process_sql_query(question, prompt, db_url):
    """
    SQL 쿼리 생성, 실행, 결과 해석의 통합 함수
    
    Args:
        question (str): 한국어 자연어 질문
        prompt (list): SQL 변환 프롬프트
        db_url (str): 데이터베이스 연결 URL
    
    Returns:
        tuple: (SQL 쿼리, 데이터프레임, 자연어 설명)
    """
    # SQL 쿼리 생성
    query = get_gemini_sql(question, prompt)
    
    # 데이터베이스 연결 및 쿼리 실행
    engine = create_engine(db_url)
    df = pd.read_sql(query, engine)

    natural_language_result = convert_result_to_natural_language(question, df)
    
    return natural_language_result

# 메인 함수
def ask(question):
    prompt = [
    """
    You are an expert in converting Korean natural language queries to precise SQL queries!

    데이터베이스 정보:
    - 데이터베이스 이름: service
    - 테이블들: gen_stat_daily, gen_stat_monthly, gen_stat_yearly

    테이블 및 컬럼 매핑 규칙:
    1. 사이트 번호 관련:
    - "사이트 번호" = site_no
    - "발전소 번호" = site_no

    2. 발전량 관련:
    - "발전량" = gen_1day (일간), gen_1month (월간), gen_1year (연간)
    - "1일 발전량" = gen_1day
    - "1달 발전량" = gen_1month
    - "1년 발전량" = gen_1year

    3. 날짜/시간 관련:
    - "날짜" = date
    - "년도" = year
    - "년" = year
    - "월" = month
    - "달" = month
    - "발전 시간" = gentime
    - "하루 평균 발전 시간" = gentime_1day_mean
    - "1일 발전 시간 평균" = gentime_1day_mean

    4. 기상 데이터 관련:
    - "누적일사량" = radiation
    - "일사량" = radiation
    - "하루 평균 일사량" = radiation_1day_mean
    - "하루 평균 누적일사량" = radiation_1day_mean
    - "1일 누적일사량 평균" = radiation_1day_mean
    - "운량" = cloud_cover
    - "하루 평균 운량" = cloud_cover_1day_mean
    - "강수량" = precipitation
    - "누적강수량" = precipitation
    - "강수일수" = precipitation_1day_count
    - "기온" = air_temperature
    - "평균 기온" = air_temperature_1day_mean
    - "하루 평균 기온" = air_temperature_1day_mean

    쿼리 변환 가이드라인:
    1. 사용자의 자연어 쿼리를 정확한 SQL 쿼리로 변환하세요.
    2. 적절한 테이블(daily/monthly/yearly)을 선택하세요.
    3. 날짜, 집계 함수 등을 정확히 고려하세요.
    4. 쿼리는 명확하고 간결해야 합니다.

    예시:
    - 입력: "2023년 사이트 3의 총 발전량은?"
      출력: SELECT gen_1year FROM service.gen_stat_yearly WHERE site_no = 3 AND year = 2023

    - 입력: "6월의 평균 일사량이 가장 높은 사이트는?"
      출력: SELECT site_no, AVG(radiation_1day_mean) as avg_radiation 
            FROM service.gen_stat_monthly 
            WHERE month = 6 
            GROUP BY site_no 
            ORDER BY avg_radiation DESC 
            LIMIT 1

    출력하는 SQL 쿼리는 반드시 다음 조건을 만족해야 합니다:
    1. 표준 SQL 문법을 따를 것
    2. 쿼리 결과가 명확할 것
    3. 불필요한 데이터는 제외할 것
    4. 가능한 한 간결하고 효율적일 것
    """
    ]
    
    db_url = "postgresql://jiu:password@ip:port/database_name"
    
    result = process_sql_query(question, prompt, db_url)
    print(result)

In [ ]:
ask('8월의 평균 일사량이 가장 높은 발전소는?')